# 📰 Tarea 1: Sistema de recuperación de noticias (RPP)


##### Instalar requirements:

In [3]:
#pip install -r requirements.txt

#### Cargar datos :

In [4]:
import feedparser
import pandas as pd


# URL del RSS de RPP
rss_url = "https://rpp.pe/rss"

feed = feedparser.parse(rss_url)

# Se extrae las 50 noticias más recientes
news_items = []
for entry in feed.entries[:50]:
    news_items.append({
        "title": entry.title,
        "description": entry.description,
        "link": entry.link,
        "date_published": entry.published if "published" in entry else None
    })

df_rpp = pd.DataFrame(news_items)
print(f"Noticias extraídas: {len(df_rpp)}")
df_rpp.head()


Noticias extraídas: 50


,title,description,link,date_published
0,Historial de reportes de sismos magnitud y epi...,¿Cuál fue el último Temblor en Chile hoy 17 de...,https://rpp.pe/mundo/chile/temblor-en-chile-ho...,"Wed, 15 Oct 2025 17:09:45 -0500"
1,Temblor en Chile hoy 17 de octubre: Epicentro ...,¿Cuál fue el último Temblor en Chile hoy 17 de...,https://rpp.pe/mundo/chile/temblor-en-chile-ho...,"Fri, 17 Oct 2025 20:09:45 -0500"
2,Cúal fue el último temblor en México hoy 17 de...,Cuál es el ultimo temblor en México y CDMX reg...,https://rpp.pe/mundo/mexico/cual-fue-el-ultimo...,"Thu, 16 Oct 2025 06:48:35 -0500"
3,"Imagine Dragons en Lima: setlist, horarios y t...",La banda de pop rock llegará por primera vez a...,https://rpp.pe/musica/conciertos/imagine-drago...,"Fri, 17 Oct 2025 19:37:24 -0500"
4,Señor de los Milagros: fe y tradición que se m...,"Cada octubre, innumerables fieles acompañan al...",https://rpp.pe/peru/actualidad/senor-de-los-mi...,"Fri, 17 Oct 2025 19:28:45 -0500"


 #### Tokenización :

In [5]:
import tiktoken

# Elegimos un modelo de referencia
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Tomamos un artículo de ejemplo
sample_text = df_rpp["description"][0]
num_tokens = len(encoding.encode(sample_text))

print(f"Número de tokens en el primer artículo: {num_tokens}")

# Si el texto supera 1024 tokens, lo dividimos
def dividir_texto_si_excede(texto, max_tokens=1024):
    tokens = encoding.encode(texto)
    if len(tokens) <= max_tokens:
        return [texto]  # No necesita dividirse
    else:
        partes = []
        for i in range(0, len(tokens), max_tokens):
            sub_tokens = tokens[i:i + max_tokens]
            sub_texto = encoding.decode(sub_tokens)
            partes.append(sub_texto)
        return partes

# Aplicamos la función al texto de ejemplo
if num_tokens > 1024:
    partes = dividir_texto_si_excede(sample_text)
    print(f"El texto se dividió en {len(partes)} fragmentos.")
else:
    print("No fue necesario dividir el texto.")

Número de tokens en el primer artículo: 53
No fue necesario dividir el texto.


 #### Generar incrustaciones (embeddings) :

In [ ]:
from sentence_transformers import SentenceTransformer

model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedder = SentenceTransformer(model_name)

# Crear las incrustaciones
embeddings = embedder.encode(df_rpp["description"].tolist(), show_progress_bar=True)

print(f"Forma de las incrustaciones: {embeddings.shape}")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\atara\anaconda3\envs\yelp_scraper\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\atara\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Forma de las incrustaciones: (50, 384)


 #### Crear base de datos vectorial con ChromaDB

In [ ]:
import chromadb
from chromadb.utils import embedding_functions

chroma_client = chromadb.Client()

# Crear o conectar a una colección
collection = chroma_client.get_or_create_collection(name="rpp_news")

# Insertar los documentos y sus metadatos
collection.add(
    documents=df_rpp["description"].tolist(),
    embeddings=embeddings,
    metadatas=[{
        "title": t,
        "link": l,
        "date_published": d
    } for t, l, d in zip(df_rpp["title"], df_rpp["link"], df_rpp["date_published"])],
    ids=[f"id_{i}" for i in range(len(df_rpp))]
)

print(" Noticias insertadas correctamente en ChromaDB")

#### Consultas de similitud: 

In [ ]:
query = "Últimas noticias de economía"

query_embedding = embedder.encode([query])
results = collection.query(
    query_embeddings=query_embedding,
    n_results=5
)

# Mostrar los resultados en un DataFrame
results_df = pd.DataFrame({
    "title": [m["title"] for m in results["metadatas"][0]],
    "description": results["documents"][0],
    "link": [m["link"] for m in results["metadatas"][0]],
    "date_published": [m["date_published"] for m in results["metadatas"][0]]
})

print("Resultados de la búsqueda por similitud:")
results_df


#### Orquestación con LangChain :


In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms.fake import FakeListLLM  # Simulación ligera sin API

# Solo simularemos un flujo completo (sin necesidad de API de OpenAI)
template = """
Dada la siguiente descripción de noticia: "{description}"
Responde con una categoría temática general (por ejemplo: política, deportes, economía, sociedad, tecnología).
"""
prompt = PromptTemplate(input_variables=["description"], template=template)

# Creamos un LLM simulado (podrías cambiarlo por ChatOpenAI si tienes API key)
fake_llm = FakeListLLM(responses=["economía", "política", "sociedad", "tecnología", "deportes"])

chain = LLMChain(llm=fake_llm, prompt=prompt)

# Probar con una noticia
test_description = df_rpp["description"][1]
chain.run(description=test_description)

print("Código ejecutado correctamente")